<a href="https://colab.research.google.com/github/lucassverass/pipedream/blob/master/projetobigdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import yfinance as yf
import tkinter as tk
from tkinter import ttk
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import warnings
import pandas as pd
import mplfinance as mpf

# Ignorar avisos
warnings.filterwarnings("ignore")

# Dicionário com os nomes completos das commodities e suas unidades
commodity_names = {
    'CC=F': 'Cacau (USD/tonelada)',
    'KC=F': 'Café (USD/libra-peso)',
    'SB=F': 'Açúcar (USD/libra-peso)',
    'CT=F': 'Algodão (USD/libra-peso)',
    'ZS=F': 'Soja (centavos de USD/bushel)',
    'LE=F': 'Carne Bovina (USD/libra-peso)',
    'TIO=F': 'Minério de Ferro (USD/tonelada)'
}

# Função para calcular o RSI
def calculate_rsi(data, window=14):
    delta = data['Close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    data['RSI'] = rsi
    return data

# Função para obter dados do Yahoo Finance com velas diárias
def get_data(tickers, interval="1d"):
    data = {}
    for ticker in tickers:
        stock_data = yf.Ticker(ticker)
        df = stock_data.history(period="1y", interval=interval)
        if df.empty:
            print(f"Nenhum dado retornado para {ticker}.")
        else:
            data[ticker] = df[['Open', 'High', 'Low', 'Close', 'Volume']]
    return data

# Função para plotar gráficos de velas com volume e RSI
def plot_candlestick(data, title, frame):
    for widget in frame.winfo_children():
        widget.destroy()

    for ticker, prices in data.items():
        prices = calculate_rsi(prices)  # Calcula o RSI antes de plotar

        # Criar um novo subplot com um maior número de candles visíveis
        fig, axes = mpf.plot(prices, type='candle', volume=True, show_nontrading=True,
                              title=title, style='charles', returnfig=True)

        # Ajustar o limite do eixo x para mostrar mais velas
        ax_candle = fig.axes[0]
        ax_candle.set_xlim(prices.index[-40], prices.index[-1])  # Mostra as últimas 40 velas para um melhor zoom

        # Adicionar o gráfico de RSI
        ax_rsi = fig.add_subplot(3, 1, 3)
        ax_rsi.plot(prices.index, prices['RSI'], color='orange', label='RSI')
        ax_rsi.axhline(70, color='red', linestyle='--', label='Overbought')
        ax_rsi.axhline(30, color='green', linestyle='--', label='Oversold')
        ax_rsi.set_ylabel('RSI')
        ax_rsi.set_ylim(0, 100)  # Limitar o eixo y do RSI

        # Ajustar o tamanho do gráfico de RSI
        ax_rsi.set_aspect(0.5)  # Reduz a altura do gráfico do RSI

        # Ajustar layout
        plt.tight_layout()
        plt.subplots_adjust(hspace=0.3)  # Espaço entre os gráficos

        # Obter o último preço de fechamento
        last_price = prices['Close'][-1]

        # Coloca o nome do ativo e o preço no canto superior esquerdo do gráfico
        ax_candle.text(0.01, 0.95, f"{ticker}: {last_price:.2f}", transform=ax_candle.transAxes, fontsize=12,
                       color='black', ha='left', va='top', bbox=dict(facecolor='white', alpha=0.7))

        # Adicionar legenda ao gráfico de RSI
        ax_rsi.legend(loc='upper left')

        # Criar o canvas para o gráfico
        canvas = FigureCanvasTkAgg(fig, master=frame)
        canvas.draw()
        canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)

# Função para plotar gráficos de linhas com bordas arredondadas
def plot_line(data, title, frame, labels=None, unit=''):
    for widget in frame.winfo_children():
        widget.destroy()

    fig, ax = plt.subplots(figsize=(10, 5))

    for ticker, prices in data.items():
        if not prices.empty:  # Verifica se o DataFrame não está vazio
            last_price = prices['Close'][-1]  # Obtém a última cotação
            label = f"{labels[ticker]}: {last_price:.2f} {unit}" if labels and ticker in labels else f"{ticker}: {last_price:.2f} {unit}"  # Inclui o valor e unidade no label
            ax.plot(prices.index, prices['Close'], label=label)

    ax.set_title(title + f" ({unit})")
    ax.legend()

    # Adiciona régua de preços à direita
    ax.yaxis.set_ticks_position('right')
    ax.yaxis.set_label_position("right")

    # Adiciona um retângulo com bordas arredondadas
    rect = patches.FancyBboxPatch((0, 0), 1, 1, boxstyle="round,pad=0.03", color="lightgrey", transform=ax.transAxes, zorder=-1)
    ax.add_patch(rect)

    fig.suptitle(title, fontsize=10)
    canvas = FigureCanvasTkAgg(fig, master=frame)
    canvas.draw()
    canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)

# Função para configurar a interface
def setup_interface(root):
    root.title("Análise de Big Data - Yahoo Finance (Gráficos de Velas)")
    root.geometry("1600x900")

    # Cria um notebook para as abas
    notebook = ttk.Notebook(root)
    notebook.pack(fill=tk.BOTH, expand=True)

    # Aumenta o tamanho da fonte das abas
    style = ttk.Style()
    style.configure("TNotebook.Tab", font=("Arial", 12))
    style.configure("TNotebook", background="lightgray")

    # Estilo para as abas quando selecionadas
    style.map("TNotebook.Tab", background=[("selected", "green"), ("active", "lightgreen")])

    # Estilo para bordas e arredondamento
    style.configure("TNotebook", borderwidth=5)
    style.configure("TNotebook.Tab", borderwidth=2, relief="solid")

    # 1. Commodities, Bancos Nacionais e Ações (Unidos)
    commodities = ['CC=F', 'KC=F', 'SB=F', 'CT=F', 'ZS=F', 'LE=F', 'TIO=F']
    commodities_data = get_data(commodities)

    bancos = ['ITUB4.SA', 'BBDC3.SA', 'BBDC4.SA', 'BBAS3.SA', 'SANB11.SA',
              'BPAC11.SA', 'BMGB4.SA', 'BRSR6.SA', 'CVCB3.SA', 'PSSA3.SA']
    bancos_data = get_data(bancos)

    # Gráficos das Ações
    brasil_stocks = ['PETR4.SA', 'VALE3.SA', 'ITUB4.SA', 'B3SA3.SA', 'MGLU3.SA']
    brasil_stocks_data = get_data(brasil_stocks)

    usa_stocks = ['AAPL', 'AMZN', 'GOOGL', 'TSLA', 'MSFT']
    usa_stocks_data = get_data(usa_stocks)

    combined_frame = tk.Frame(notebook)
    notebook.add(combined_frame, text='Commodities, Bancos e Ações')

    # Dividir o frame em uma grade 2x2
    left_frame_top = tk.Frame(combined_frame)
    left_frame_top.grid(row=0, column=0, padx=10, pady=10, sticky='nsew')
    left_frame_bottom = tk.Frame(combined_frame)
    left_frame_bottom.grid(row=1, column=0, padx=10, pady=10, sticky='nsew')
    right_frame_top = tk.Frame(combined_frame)
    right_frame_top.grid(row=0, column=1, padx=10, pady=10, sticky='nsew')
    right_frame_bottom = tk.Frame(combined_frame)
    right_frame_bottom.grid(row=1, column=1, padx=10, pady=10, sticky='nsew')

    combined_frame.grid_rowconfigure(0, weight=1)
    combined_frame.grid_rowconfigure(1, weight=1)
    combined_frame.grid_columnconfigure(0, weight=1)
    combined_frame.grid_columnconfigure(1, weight=1)

    # Gráficos de Commodities
    plot_line(commodities_data, 'Principais Commodities', left_frame_top, labels=commodity_names, unit='USD')

    # Gráficos de Bancos Nacionais
    plot_line(bancos_data, 'Principais Bancos Nacionais', left_frame_bottom, unit='BRL')

    # Gráficos de Ações Brasileiras
    plot_line(brasil_stocks_data, 'Principais Ações Brasileiras', right_frame_top, unit='BRL')

    # Gráficos de Ações Americanas
    plot_line(usa_stocks_data, 'Principais Ações Americanas', right_frame_bottom, unit='USD')

    # 2. Índices: Bovespa e S&P 500
    indices_frame = tk.Frame(notebook)
    notebook.add(indices_frame, text='Índices: Bovespa e S&P 500')

    # Dividindo o frame em dois subframes
    bovespa_frame = tk.Frame(indices_frame)
    bovespa_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=10, pady=10)
    sp500_frame = tk.Frame(indices_frame)
    sp500_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True, padx=10, pady=10)

    # Índice Bovespa
    indices_bovespa = ['^BVSP']
    bovespa_data = get_data(indices_bovespa)
    plot_candlestick(bovespa_data, 'Gráfico do Índice Bovespa', bovespa_frame)

    # Índice S&P 500
    indices_sp500 = ['^GSPC']
    sp500_data = get_data(indices_sp500)
    plot_candlestick(sp500_data, 'Gráfico do S&P 500', sp500_frame)

    # 3. Principais corretoras estrangeiras
    brokers_frame = tk.Frame(notebook)
    notebook.add(brokers_frame, text='Corretoras Estrangeiras')

    # Corretoras Brasileiras
    brokers_brazil = ['XPBR31.SA', 'CASH3.SA', 'BPAC11.SA', 'RICO3.SA', 'MODL11.SA']  # Adicionei os tickers
    brokers_brazil_data = get_data(brokers_brazil)

    # Corretoras Estrangeiras
    brokers_foreign = ['MS', 'UBS', 'JPM', 'MERR', 'GS']
    brokers_foreign_data = get_data(brokers_foreign)

    # Dividir o frame de corretoras em 2
    frame_left = tk.Frame(brokers_frame)
    frame_left.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=10, pady=10)
    frame_right = tk.Frame(brokers_frame)
    frame_right.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True, padx=10, pady=10)

    # Gráficos das corretoras brasileiras
    plot_line(brokers_brazil_data, 'Corretoras Brasileiras', frame_left, unit='BRL')

    # Gráficos das corretoras estrangeiras
    plot_line(brokers_foreign_data, 'Corretoras Estrangeiras', frame_right, unit='USD')

    # 4. Principais criptomoedas
    cryptos = ['BTC-USD', 'ETH-USD', 'XRP-USD', 'LTC-USD', 'BCH-USD']
    cryptos_data = get_data(cryptos)
    cryptos_frame = tk.Frame(notebook)
    notebook.add(cryptos_frame, text='Principais Criptomoedas')
    plot_line(cryptos_data, 'Principais Criptomoedas', cryptos_frame, unit='USD')

        # 4. Adicionar nova aba para Notícias Futuras
    noticias_frame = tk.Frame(notebook)
    notebook.add(noticias_frame, text='Notícias Futuras')

    # Adicionar uma tabela ou informações de notícias futuras (exemplo fictício)
    news_label = tk.Label(noticias_frame, text="Notícias futuras relacionadas ao mercado", font=("Arial", 16))
    news_label.pack(pady=10)

    news_info = [
        "10:00 - PIB do Brasil",
        "14:30 - Decisão sobre a taxa de juros nos EUA",
        "16:00 - Lucros da Apple"
    ]

    for news in news_info:
        news_label = tk.Label(noticias_frame, text=news, font=("Arial", 14))
        news_label.pack(anchor='w')


    return notebook

# Função principal para execução da interface
def main():
    root = tk.Tk()
    setup_interface(root)

    root.mainloop()

if _name_ == "_main_":
    main()


NameError: name '_name_' is not defined